In [ ]:
# Autoreload modules
%load_ext autoreload
%autoreload 2

In [ ]:
# To have access to moduels
import sys,os
sys.path.append(os.path.dirname(os.path.realpath('')) + '/Modules')

In [ ]:
import numpy as np

import torch
import torch.nn as nn
from transformers import ViTConfig, ViTFeatureExtractor, ViTForImageClassification

from matplotlib import pyplot as plt

import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from dataloader.dataset import ADNI
from dataloader.dataloader import ADNILoader

from utils.report import sklearn_classification_report

# Dataset and Dataloader Setup

In [ ]:
image_size = (79, 95)

train_ds = ADNI("../Data/Training/", transforms=None, extra_channel_dim=False, rotate=False)
valid_ds = ADNI("../Data/Validation/", transforms=None, extra_channel_dim=False, rotate=False)
test_ds = ADNI("../Data/Test/", transforms=None, extra_channel_dim=False, rotate=False)

In [ ]:
idx = 0
image, label = train_ds[idx]

print("Image shape:", image.shape)
print("Label:", label.item())

print("Number of training samples:", len(train_ds))
print("Number of validation samples:", len(valid_ds))
print("Number of test samples:", len(test_ds), "\n")

fig, axes = plt.subplots(nrows=6, ncols=10, figsize=(3, 2), dpi=300)
for row in range(6):
    for col in range(10):
        idx = row * 10 + col
        axes[row, col].imshow(image[idx, :, :])
        axes[row, col].axis("off");
        # print(image[idx, :, :].min(), image[idx, :, :].max())

In [ ]:
id2label = {0: "CN", 1: "MCI", 2: "AD"}
label2id = {"CN": 0, "MCI": 1, "AD": 2}

print(id2label[label.item()])

In [ ]:
train_batch_size = 15
valid_batch_size = 10
test_batch_size = 10

hparams = {'train_ds': train_ds,
           'valid_ds': valid_ds,
           'test_ds': test_ds,
           'train_batch_size': train_batch_size,
           'valid_batch_size': valid_batch_size,
           'test_batch_size': test_batch_size,
           'num_workers': 20,
           'train_shuffle': False,
           'valid_shuffle': False,
           'test_shuffle': False,
           'train_drop_last': True,
           'valid_drop_last': False,
           'test_drop_last': False,
          }

train_dataloader = ADNILoader(**hparams).train_dataloader()
valid_dataloader= ADNILoader(**hparams).validation_dataloader()
test_dataloader = ADNILoader(**hparams).test_dataloader()

batch = next(iter(train_dataloader))
print(batch[0].shape)
print(batch[1].shape)

# Model Development

In [ ]:
vit_config = ViTConfig(image_size=image_size,
                       patch_size=32,
                       num_labels=3,
                       num_channels=60,
                       num_hidden_layers=12,
                       hidden_size=768,
                       intermediate_size=3072,
                       num_attention_heads=12,
                       output_attentions=True,
                       output_hidden_states=True,
                       hidden_dropout_prob=0,
                       attention_probs_dropout_prob=0,
                      )

In [ ]:
class ViT(nn.Module):
    def __init__(self, num_labels=3, return_last_hidden_state=False):
        super(ViT, self).__init__()
        self.return_last_hidden_state = return_last_hidden_state
        self.vit = ViTForImageClassification(vit_config)
        
    def forward(self, x):
        outputs = self.vit(x)
        if self.return_last_hidden_state:
            return outputs.logits, outputs.attentions, outputs.hidden_states
        else:
            return outputs.logits, outputs.attentions

# Saving Hidden States

In [ ]:
device = torch.device('cpu') 
model = ViT(return_last_hidden_state=False).to(device)
feature_extractor = ViTFeatureExtractor(size=image_size,
                                        do_resize=False,
                                        do_normalize=False)

model.load_state_dict(torch.load("../ViT/Best models/ViT_3D_acc.pt"))

In [ ]:
def predict(model, dataloader, device):
    y_true = []
    y_pred = []
    
    model.eval()
    with torch.no_grad():
        for step, (x, y) in enumerate(dataloader):
            x = np.split(np.array(x), dataloader.batch_size)
            for i in range(len(x)):
                x[i] = np.squeeze(x[i])
            x = torch.tensor(np.stack(feature_extractor(x)['pixel_values'], axis=0))
            x, y  = x.to(device), y.to(device)
            logits, _ = model(x)
            preds = logits.argmax(1)
        
            y_pred.append(preds.cpu().numpy())
            y_true.append(y.cpu().numpy())

    y_pred = np.concatenate(y_pred, axis=0)
    y_true = np.concatenate(y_true, axis=0)
    
    return y_true, y_pred

y_true, y_pred = predict(model, test_dataloader, device)
sklearn_classification_report(y_true, y_pred)

# Preparing Activation and Label Dataframes

In [ ]:
x_train_df = pd.DataFrame()
y_train_df = pd.DataFrame()

x_test_df = pd.DataFrame()
y_test_df = pd.DataFrame()

model.return_last_hidden_state = True
model.eval()

with torch.no_grad():
    for sample, label in train_dataloader:
        sample = np.split(np.array(sample), train_batch_size)
        for i in range(len(sample)):
            sample[i] = np.squeeze(sample[i])
        sample = torch.tensor(np.stack(feature_extractor(sample)['pixel_values'], axis=0))
        sample, label  = sample.to(device), label.to(device)
        _, _, activation = model(sample)

        act_df = pd.DataFrame(activation[12].reshape((train_batch_size, 5*768)).detach().numpy())
        label_df = pd.DataFrame(label.detach().numpy())

        x_train_df = pd.concat([x_train_df, act_df])
        y_train_df = pd.concat([y_train_df, label_df])

with torch.no_grad():
    for sample, label in test_dataloader:
        sample = np.split(np.array(sample), test_batch_size)
        for i in range(len(sample)):
            sample[i] = np.squeeze(sample[i])
        sample = torch.tensor(np.stack(feature_extractor(sample)['pixel_values'], axis=0))
        sample, label  = sample.to(device), label.to(device)
        _, _, activation = model(sample)

        act_df = pd.DataFrame(activation[12].reshape((test_batch_size, 5*768)).detach().numpy())
        label_df = pd.DataFrame(label.detach().numpy())

        x_test_df = pd.concat([x_test_df, act_df])
        y_test_df = pd.concat([y_test_df, label_df])

# PCA

In [ ]:
x_train_df = x_train_df.reset_index(drop=True)
y_train_df = y_train_df.rename(columns={0: "target"}).reset_index(drop=True)
x_test_df = x_test_df.reset_index(drop=True)
y_test_df = y_test_df.rename(columns={0: "target"}).reset_index(drop=True)

scaler = StandardScaler()
x_train_std = scaler.fit_transform(x_train_df)
x_test_std = scaler.fit_transform(x_test_df)

pca = PCA(n_components=2)
pca_train = pca.fit_transform(x_train_std)
pca_test = pca.fit_transform(x_test_std)

pca_train_df = pd.DataFrame(pca_train, columns = ['principal component 1', 'principal component 2'])
pca_test_df = pd.DataFrame(pca_test, columns = ['principal component 1', 'principal component 2'])

final_train_df = pd.concat([pca_train_df, y_train_df], axis = 1)
final_test_df = pd.concat([pca_test_df, y_test_df], axis = 1)

final_df = pd.concat([final_train_df, final_test_df], axis = 0, keys=['train', 'test'])
final_df

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8), dpi=300)
targets = [0, 1, 2]
colors = ['r', 'g', 'b']

for i in range(len(targets)):
    sample_df = final_df[final_df['target'] == targets[i]]
    
    ax.scatter(sample_df.loc['train', 'principal component 1'],
               sample_df.loc['train', 'principal component 2'],
               s=150,
               alpha=0.3,
               c=colors[i],
               marker='o',
               label=f'{id2label[targets[i]]} (Train)'
              )
    
    ax.scatter(sample_df.loc['test', 'principal component 1'],
               sample_df.loc['test', 'principal component 2'],
               s=150,
               alpha=0.3,
               c=colors[i],
               marker='*',
               label=f'{id2label[targets[i]]} (Test)'
              )
    
ax.legend();
ax.set_xlabel('Principle Component 1')
ax.set_ylabel('Principle Component 2');
# plt.savefig('PCA_ViT_3D.png')